## Simple Mission in Auto-Mode

This notebook serves to check the ardupilot installation

In [1]:
import os
import time

from config import (
    ARDU_LOGS_PATH,
    ARDUPILOT_VEHICLE_PATH,
    ENV_CMD_PYT,
    VEH_PARAMS_PATH,
    BasePort,
)
from helpers import clean, create_process, setup_logging
from mavlink.util import connect
from plan import State
from plan.planner import Plan

clean()


## Launch Copter (ardupilot)

In [2]:
#This must agree with first waypoint in mission.waypoint
spawn_str= '-35.3633245,149.1652241,0.0,0' 

sim_vehicle_path = os.path.expanduser(ARDUPILOT_VEHICLE_PATH)
vehicle_cmd = (
    f"python3 {sim_vehicle_path} "
    f"-v ArduCopter "
    f"-I0 --sysid 1 "
    f"--no-rebuild "
    f"--use-dir={ARDU_LOGS_PATH} "
    f"--add-param-file {VEH_PARAMS_PATH} "
    f"--no-mavproxy "
    f'--custom-location={spawn_str}'
)


create_process(
                vehicle_cmd,
                after="exec bash",
                visible= True,
                suppress_output=False,
                title="ardu_vehicle",
                env_cmd=ENV_CMD_PYT,
            ) 


<Popen: returncode: None args: ['gnome-terminal', '--title', 'ardu_vehicle',...>

## 1. Connect to the vehicle

In [3]:
time.sleep(1)
conn = connect(f"tcp:127.0.0.1: {BasePort.ARP}")
conn.wait_heartbeat()
print("✅ Heartbeat received")


✅ Heartbeat received


## Create auto plan from mission file

The mission is loaded from "plan/missions/simple_mission.waypoints"

In [4]:
plan = Plan.auto(name='auto',
                mission_path="plan/missions/simple_mission.waypoints",
                monitored_items=[1,2])
plan

🕓 <Plan '📋 auto'>
  🕓 <Action '📤 UPLOAD_MISSION'>
    🕓 <Step '🔹 clear uav missions'>
    🕓 <Step '🔹 load mission plan/missions/simple_mission.waypoints'>
  🕓 <Action '🔧 PREARM'>
    🕓 <Step '🔹 Check disarmed'>
    🕓 <Step '🔹 Check EKF status'>
    🕓 <Step '🔹 Check GPS'>
    🕓 <Step '🔹 Check system'>
  🕓 <Action '⚙️  MODE: GUIDED'>
    🕓 <Step '🔹 Switch to GUIDED'>
  🕓 <Action '🔐 ARM'>
    🕓 <Step '🔹 arm'>
  🕓 <Action '🚀 START_MISSION'>
    🕓 <Step '🔹 start_mission'>
  🕓 <Action '👁️ MONITOR_MISSION'>
    🕓 <Step '🔹 check item 1'>
    🕓 <Step '🔹 check item 2'>
    🕓 <Step '🔹 check mission completion'>

# Execute Auto Plan

In [5]:
plan.bind(conn)
setup_logging('plan',verbose=2)
while plan.state != State.DONE:
    plan.act()

15:53:56 - plan - DEBUG - ▶️ Vehicle 1: Plan Started: 📋 auto
15:53:56 - plan - DEBUG - ▶️ Vehicle 1: Action Started: 📤 UPLOAD_MISSION
15:53:56 - plan - DEBUG - ▶️ Vehicle 1: Step Started: clear uav missions
15:53:58 - plan - INFO - 🧹 Vehicle 1: Cleared previous mission
15:53:58 - plan - DEBUG - ✅ Vehicle 1: Step Done: clear uav missions
15:53:58 - plan - INFO - ✅ Vehicle 1: 4 waypoints read
15:53:58 - plan - DEBUG - 🧭 Vehicle 1: Mission[0] → cmd: WAYPOINT, x: -35.3633245, y: 149.1652241, z: 0.0, current: 0
15:53:58 - plan - DEBUG - 🧭 Vehicle 1: Mission[1] → cmd: TAKEOFF, x: -35.3633245, y: 149.1652241, z: 5.0, current: 0
15:53:58 - plan - DEBUG - 🧭 Vehicle 1: Mission[2] → cmd: WAYPOINT, x: -35.3632795, y: 149.1652241, z: 5.0, current: 0
15:53:58 - plan - DEBUG - 🧭 Vehicle 1: Mission[3] → cmd: LAND, x: -35.3632795, y: 149.1652241, z: 0.0, current: 0
15:53:59 - plan - DEBUG - ✅ Vehicle 1: Sent mission item 0
15:53:59 - plan - DEBUG - ✅ Vehicle 1: Sent mission item 1
15:53:59 - plan - DEB

In [6]:
plan

✅ <Plan '📋 auto'>
  ✅ <Action '📤 UPLOAD_MISSION'>
    ✅ <Step '🔹 clear uav missions'>
    ✅ <Step '🔹 load mission plan/missions/simple_mission.waypoints'>
  ✅ <Action '🔧 PREARM'>
    ✅ <Step '🔹 Check disarmed'>
    ✅ <Step '🔹 Check EKF status'>
    ✅ <Step '🔹 Check GPS'>
    ✅ <Step '🔹 Check system'>
  ✅ <Action '⚙️  MODE: GUIDED'>
    ✅ <Step '🔹 Switch to GUIDED'>
  ✅ <Action '🔐 ARM'>
    ✅ <Step '🔹 arm'>
  ✅ <Action '🚀 START_MISSION'>
    ✅ <Step '🔹 start_mission'>
  ✅ <Action '👁️ MONITOR_MISSION'>
    ✅ <Step '🔹 check item 1'>
    ✅ <Step '🔹 check item 2'>
    ✅ <Step '🔹 check mission completion'>